In [ ]:
%cd /mnt/87433d53-66f3-438e-a7f4-fc990fe61283/PROJECTS/yolov5/

In [ ]:
import os
import torch
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image, clear_output
import warnings
warnings.filterwarnings("ignore")

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

# Inference

In [ ]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'custom', path = '/mnt/87433d53-66f3-438e-a7f4-fc990fe61283/PROJECTS/yolov5/runs/train/exp3/weights/best.pt')  # or yolov5m, yolov5l, yolov5x, custom

In [ ]:
results = model('/mnt/036b1314-13ad-429f-b9f9-07467946c5d7/Datasets/Health/Leukomia/ALL-IDB/train/Im002_1.jpg')
coordinates = results.xyxy[0].cpu().numpy()
coordinates

# Draw YOLO predicted bounding boxes

In [ ]:
test_dir = '/mnt/036b1314-13ad-429f-b9f9-07467946c5d7/Datasets/Health/Leukomia/ALL-IDB/test-from-train'
output_dir = '/mnt/036b1314-13ad-429f-b9f9-07467946c5d7/Datasets/Health/Leukomia/ALL-IDB/outputs'

for filename in os.listdir(test_dir):
    file = os.path.join(test_dir, filename)
#     img = cv2.imread(file)
    results = model(file)

    coordinates = results.xyxy[0].cpu().numpy()
    
    im = cv2.imread(file)
    for i in coordinates:
        start = (int(i[0]),int(i[1]))
        
        end = (int(i[2]),int(i[3]))
        cv2.rectangle(im, start, end, (255,0,0), 2)

    cv2.imwrite(os.path.join(output_dir, filename), im)

In [ ]:
!pwd

In [ ]:
import torch

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Image
im = 'https://ultralytics.com/images/zidane.jpg'

# Inference
results = model(im)

results.pandas().xyxy[0]
#      xmin    ymin    xmax   ymax  confidence  class    name
# 0  749.50   43.50  1148.0  704.5    0.874023      0  person
# 1  433.50  433.50   517.5  714.5    0.687988     27     tie
# 2  114.75  195.75  1095.0  708.0    0.624512      0  person
# 3  986.00  304.00  1028.0  420.0    0.286865     27     tie

In [ ]:
# Model
wts_path = '/mnt/87433d53-66f3-438e-a7f4-fc990fe61283/PROJECTS/yolov5/runs/train/exp5/weights/best.pt'
model = torch.hub.load('ultralytics/yolov5', 'custom', path = wts_path)

for filename in os.listdir(raw_img_dir):
    file = os.path.join(raw_img_dir, filename)
    
    results = model(file)

    coordinates = results.xyxy[0].cpu().numpy()
    
    im = cv2.imread(file)
    for i in coordinates:
        start = (int(i[0]),int(i[1]))
        
        end = (int(i[2]),int(i[3]))
        cv2.rectangle(im, start, end, (255,0,0), 2)

    cv2.imwrite(os.path.join(output_dir, filename), im)

# .tif to .jpg

In [ ]:
for filename in os.listdir(output_dir):
    if '.tif' in filename:
        file = os.path.join(output_dir, filename)
        print(file)
        img = cv2.imread(file)
        img = cv2.imwrite(os.path.join(output_dir, filename[:-3]+'jpg'), img)
        os.remove(file)

# Compare

In [ ]:
comparison_dir = os.path.join(data_root_dir, 'Comparison')

for i, j, k in zip(os.listdir(raw_img_dir), os.listdir(p_a_img_dir), os.listdir(output_dir)):
    if i[-3:]  == 'jpg' and j[-3:]  == 'jpg' and k[-3:]  == 'jpg': 
        file1 = os.path.join(raw_img_dir, i)
        file2 = os.path.join(p_a_img_dir, j)
        file3 = os.path.join(output_dir, k)
        
        img1 = plt.imread(file1)
        img2 = plt.imread(file2)
        img3 = plt.imread(file3)
        
        plt.figure(figsize = (50,35))
        plt.subplots_adjust(hspace=0.5, wspace=0.1)

        plt.subplot(1, 3, 1)
        plt.xticks([])
        plt.yticks([])
        plt.title('Original image', fontsize=35)
        plt.imshow(img1)
        
        plt.subplot(1, 3, 2)
        plt.xticks([])
        plt.yticks([])
        plt.title('Pre-annotated image', fontsize=35)
        plt.imshow(img2)
        
        plt.subplot(1, 3, 3)
        plt.xticks([])
        plt.yticks([])
        plt.title('YOLO predicted image', fontsize=35)
        plt.imshow(img3)
        plt.savefig(os.path.join(comparison_dir, i), bbox_inches = 'tight')
        plt.show()